This dataset comprises 1 million professionally annotated article-summary pairs from BBC, extracted using a set of carefully designed heuristics.

In [1]:
!git clone https://github.com/AzzaAbdelGhani/xl-sum.git

Cloning into 'xl-sum'...
remote: Enumerating objects: 1148, done.
remote: Counting objects: 100% (1148/1148), done.
remote: Compressing objects: 100% (817/817), done.
remote: Total 1148 (delta 314), reused 1124 (delta 301), pack-reused 0
Receiving objects: 100% (1148/1148), 5.51 MiB | 13.50 MiB/s, done.
Resolving deltas: 100% (314/314), done.


In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:25
🔁 Restarting kernel...


In [3]:
!cd xl-sum/seq2seq
!conda create python==3.7.9 pytorch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2 cudatoolkit=10.2 -c pytorch -p ./env
!conda activate ./env # or source activate ./env (for older versions of anaconda)
!bash xl-sum/seq2seq/setup.sh 

Solving environment: - failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /content/env

  added / updated specs:
    - cudatoolkit=10.2
    - python==3.7.9
    - pytorch==1.7.1
    - torchaudio==0.7.2
    - torchvision==0.8.2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    ca-certificates-2021.10.8  |       

In [4]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1fKxf9jAj0KptzlxUsI3jDbp4XLv_piiD' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1fKxf9jAj0KptzlxUsI3jDbp4XLv_piiD" -O XLSum_complete_v2.0.tar.bz2 && rm -rf /tmp/cookies.txt
!tar -xjvf XLSum_complete_v2.0.tar.bz2
!python xl-sum/seq2seq/extract_data.py -i XLSum_complete_v2.0/ -o XLSum_input/

--2022-04-17 12:39:47--  https://docs.google.com/uc?export=download&confirm=t&id=1fKxf9jAj0KptzlxUsI3jDbp4XLv_piiD
Resolving docs.google.com (docs.google.com)... 172.253.62.101, 172.253.62.139, 172.253.62.102, ...
Connecting to docs.google.com (docs.google.com)|172.253.62.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-24-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1c1dsd2tjb0a9rt2cll1vanaojf1fgg2/1650199125000/07441260459817291408/*/1fKxf9jAj0KptzlxUsI3jDbp4XLv_piiD?e=download [following]
--2022-04-17 12:39:47--  https://doc-08-24-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1c1dsd2tjb0a9rt2cll1vanaojf1fgg2/1650199125000/07441260459817291408/*/1fKxf9jAj0KptzlxUsI3jDbp4XLv_piiD?e=download
Resolving doc-08-24-docs.googleusercontent.com (doc-08-24-docs.googleusercontent.com)... 172.217.0.33, 2607:f8b0:4004:800::2001
Connecting to doc-08-24-docs.googleusercontent.com (doc-08-24

In [1]:
!pip install jsonlines

In [3]:
import jsonlines 
import pandas as pd 
import random

languages = ['arabic','english','french','spanish', 'russian', 'portuguese']
train_list = [] #442820
test_list = [] #34222
val_list = [] #34127
for w in ['train', 'test','val']: 
  for lang in languages :
    with jsonlines.open('XLSum_complete_v2.0/{lang}_{w}.jsonl'.format(lang=lang, w=w)) as f:
      for line in f.iter():
        line['lang'] = lang
        if len(line['text']) > 1000:
          if w == 'train' : 
            train_list.append(line)
          elif w == 'test':
            test_list.append(line)
          else:
            val_list.append(line)
random.shuffle(train_list)
random.shuffle(test_list)
random.shuffle(val_list)
train_dataset = pd.DataFrame(train_list)
test_dataset = pd.DataFrame(test_list)
val_dataset = pd.DataFrame(val_list)

In [4]:
len(val_dataset)

34127

In [5]:
len(train_dataset)

442820

In [6]:
len(test_dataset)

34222

In [8]:
train_dataset.to_csv('data/train_dataset.csv')
test_dataset.to_csv('data/test_dataset.csv')
val_dataset.to_csv('data/val_dataset.csv')
#from google.colab import drive
#drive.mount('/content/gdrive')
#!cp new_train_dataset.csv "gdrive/My Drive/Dataset"
#!cp new_test_dataset.csv "gdrive/My Drive/Dataset"
#!cp new_val_dataset.csv "gdrive/My Drive/Dataset"

Mounted at /content/gdrive


In [9]:
train_dataset.head()

,id,url,title,summary,text,lang
0,uk-politics-23514804,https://www.bbc.com/news/uk-politics-23514804,The lessons and warnings from 1983 archive,It is tempting to read the newly declassified ...,"By Ross HawkinsPolitical correspondent, BBC Ne...",english
1,040714_arquitetoas,https://www.bbc.com/portuguese/noticias/story/...,Arquiteto processa arrendatário da área do WTC...,O arquiteto que criou o projeto que substituir...,"Daniel Libeskind ganhou, há 18 meses, o concur...",portuguese
2,noticias-49026735,https://www.bbc.com/mundo/noticias-49026735,Palabras polisémicas: ¿qué son y cómo las usas...,Ayer recibí una carta de mi tío Alfonso en la ...,Es probable que utilices palabras polisémicas ...,spanish
3,26539203,https://www.bbc.com/news/26539203,Gwendoline and Margaret Davies' help for WW1 r...,They are best known for amassing one of the gr...,By Charlotte DubenskijBBC Wales News They are ...,english
4,health-50290033,https://www.bbc.com/news/health-50290033,11 charts on the problems facing the NHS,The NHS faces a number of challenges and often...,By Nick Triggle and Ben ButcherBBC News 1. We ...,english


In [10]:
test_dataset.head()

,id,url,title,summary,text,lang
0,130522_woolwich_incident,https://www.bbc.com/arabic/worldnews/2013/05/1...,قاتل البريطاني في لندن يبرر فعلته أمام الكامير...,قتل شخص وجرح اثنان آخران في اعتداء استخدم فيه ...,وظهر أحد المهاجمين في فيديو يحتوي مشاهد مؤلمة ...,arabic
1,uk-northern-ireland-34164806,https://www.bbc.com/news/uk-northern-ireland-3...,Welfare reform: Westminster may legislate dire...,The Westminster government is prepared to legi...,The Northern Ireland secretary said such a mov...,english
2,100809_iraq_trafficpolice,https://www.bbc.com/arabic/middleeast/2010/08/...,تسليح شرطة مرور بغداد ببنادق اوتوماتيكية,قالت السلطات العراقية انها شرعت في تسليح افراد...,قرار التسليح بعد تكرار تعرضهم للهجمات يشار الى...,arabic
3,110921_patriota_cacas_jf,https://www.bbc.com/portuguese/noticias/2011/0...,Negociação sobre caças franceses pode ser reto...,A negociação sobre a venda de caças franceses ...,"Segundo chanceler, diálogo pode ocorrer caso a...",portuguese
4,ukrainian.141121_ru_s_press_review,https://www.bbc.com/ukrainian/ukraine_in_russi...,"Пресса: украинцам надоело ""перемирие""",21 ноября украинская пресса пишет о визите в К...,Опитування: більше третини українців виступают...,russian


In [11]:
val_dataset.head()

,id,url,title,summary,text,lang
0,ukrainian.160216_ru_s_ukr_press,https://www.bbc.com/ukrainian/ukraine_in_russi...,Тучи над правительством - обзор прессы,16 февраля украинские газеты пишут о долгождан...,Отчет правительства и путь в ЕС Уже во вторник...,russian
1,ukrainian.news-russian-40472563,https://www.bbc.com/ukrainian/news-russian-404...,Министр экологии: вопрос №1 - мониторинг окруж...,Правительство должно восстановить постоянный м...,Разрушение на Донбассе имеют серьезные экологи...,russian
2,140824_uk_hammond_foley_killing_betrayal,https://www.bbc.com/russian/uk/2014/08/140824_...,Лондон: убийцы журналиста Фоули – предатели Бр...,Министр иностранных дел Великобритании Филип Х...,Британские власти пытаются установить личность...,russian
3,uk-scotland-south-scotland-40104967,https://www.bbc.com/news/uk-scotland-south-sco...,Galloway Viking hoard artefacts to go on displ...,A selection of artefacts from a Viking treasur...,"National Museums Scotland, which saw off a Dum...",english
4,world-latin-america-41996062,https://www.bbc.com/news/world-latin-america-4...,Ecuador's Vice-President Glas to face corrupti...,A judge in Ecuador has ordered that the countr...,"Mr Glas, who was arrested on 2 October, is the...",english
